In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
import datetime
import csv
from scipy import stats as st
import itertools
import re 

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'C:\\Users\\Noah\\CovidMobile'

In [3]:
def prez_data():
    prez = pd.read_csv("data/countypres_2000-2016.csv")
    prez = prez[prez['year']==2016]
    prez['vote_share'] = prez['candidatevotes'] / prez['totalvotes']
    prez['max'] = prez.groupby(['FIPS','year'])['candidatevotes'].transform(max)
    prez_win = prez[prez['max']==prez['candidatevotes']].copy()
    prez_win = pd.concat([prez_win.reset_index(),prez[prez['party']=='republican']['vote_share'].rename('vote_share_rep').reset_index()],axis=1)
    return prez_win

In [96]:
df = pd.read_csv('data/compiled_2020-05-14.csv')
df.shape

(224170, 107)

In [97]:
['soe', 'sah', 'state_ease', 'state_sah',
'state_mandate', 'state_non_ess', 'state_lgb', 'state_sc',
'state_bar_res', 'state_election', 'state_emergency',
'state_compliance', 'state_prepare']

['soe',
 'sah',
 'state_ease',
 'state_sah',
 'state_mandate',
 'state_non_ess',
 'state_lgb',
 'state_sc',
 'state_bar_res',
 'state_election',
 'state_emergency',
 'state_compliance',
 'state_prepare']

In [98]:
df.columns.values

array(['country_region_code', 'country_region', 'state_x', 'county_x',
       'date', 'retail', 'food_drugs', 'park', 'transit', 'work', 'homes',
       'source', 'state_abb_x', 'FIPS', 'geo', 'POP_ESTIMATE_2018',
       'N_POP_CHG_2018', 'Births_2018', 'Deaths_2018', 'NATURAL_INC_2018',
       'INTERNATIONAL_MIG_2018', 'DOMESTIC_MIG_2018', 'NET_MIG_2018',
       'RESIDUAL_2018', 'GQ_ESTIMATES_2018', 'R_birth_2018',
       'R_death_2018', 'R_NATURAL_INC_2018', 'R_INTERNATIONAL_MIG_2018',
       'R_DOMESTIC_MIG_2018', 'R_NET_MIG_2018', 'POVALL_2018',
       'CI90LBAll_2018', 'CI90UBALL_2018', 'PCTPOVALL_2018',
       'CI90LBALLP_2018', 'CI90UBALLP_2018', 'POV017_2018',
       'CI90LB017_2018', 'CI90UB017_2018', 'PCTPOV017_2018',
       'CI90LB017P_2018', 'CI90UB017P_2018', 'POV517_2018',
       'CI90LB517_2018', 'CI90UB517_2018', 'PCTPOV517_2018',
       'CI90LB517P_2018', 'CI90UB517P_2018', 'MEDHHINC_2018',
       'CI90LBINC_2018', 'CI90UBINC_2018', 'POV04_2018', 'CI90LB04_2018',
     

In [99]:
dct = df[['state_abb_x','FIPS']].drop_duplicates().dropna()
dct.columns = ['neighbors_state','neighbors']


In [100]:
neighbors = df[['state_abb_x','FIPS']+list(df.filter(like='neighbors').columns.values)].drop_duplicates().dropna(subset=['state_abb_x','FIPS'])
neighbors

,state_abb_x,FIPS,neighbors1,neighbors2,neighbors3,neighbors4,neighbors5,neighbors6,neighbors7,neighbors8,neighbors9,neighbors10,neighbors11,neighbors12,neighbors13,neighbors14,neighbors15
166,AL,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,AL,1001.0,1001.0,1021.0,1047.0,1051.0,1085.0,1101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,AL,1003.0,1003.0,1025.0,1053.0,1097.0,1099.0,1129.0,12033.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,AL,1005.0,1005.0,1011.0,1045.0,1067.0,1109.0,1113.0,13061.0,13239.0,13259.0,NaN,NaN,NaN,NaN,NaN,NaN
498,AL,1007.0,1007.0,1021.0,1065.0,1073.0,1105.0,1117.0,1125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223793,WY,56037.0,8081.0,49009.0,49043.0,56007.0,56013.0,56023.0,56035.0,56037.0,56041.0,NaN,NaN,NaN,NaN,NaN,NaN
223876,WY,56039.0,16019.0,16043.0,16081.0,30031.0,56013.0,56023.0,56029.0,56035.0,56039.0,NaN,NaN,NaN,NaN,NaN,NaN
223959,WY,56041.0,49033.0,49043.0,56023.0,56037.0,56041.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224042,WY,56043.0,56003.0,56013.0,56017.0,56019.0,56025.0,56029.0,56043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
for x in range(1,16,1):
    dct.columns = ['neighbors_state{}'.format(x),'neighbors{}'.format(x)]
    neighbors = neighbors.merge(dct,on=['neighbors{}'.format(x)],how='left')

In [102]:
neighbors['border_states'] = neighbors.filter(like='neighbors_state').T.agg([pd.unique]).T
neighbors['count_border_states'] = neighbors['border_states'].apply(len)-2

In [103]:
neighbors[neighbors['count_border_states']==3]

,state_abb_x,FIPS,neighbors1,neighbors2,neighbors3,neighbors4,neighbors5,neighbors6,neighbors7,neighbors8,...,neighbors_state8,neighbors_state9,neighbors_state10,neighbors_state11,neighbors_state12,neighbors_state13,neighbors_state14,neighbors_state15,border_states,count_border_states
82,AZ,4001.0,4001.0,4009.0,4011.0,4017.0,8083.0,35003.0,35006.0,35031.0,...,NM,NM,UT,NaN,NaN,NaN,NaN,NaN,"[AZ, CO, nan, NM, UT]",3
90,AZ,4015.0,4005.0,4012.0,4015.0,4025.0,6071.0,32003.0,32017.0,49025.0,...,UT,UT,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, CA, NV, nan, UT]",3
261,CO,8083.0,4001.0,8033.0,8067.0,8083.0,8111.0,35045.0,49037.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, nan, CO, NM, UT]",3
292,DE,10003.0,10001.0,10003.0,24015.0,24029.0,34015.0,34033.0,42029.0,42045.0,...,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[DE, MD, NJ, PA, nan]",3
1128,MD,24043.0,24001.0,24021.0,24043.0,42055.0,42057.0,51107.0,54003.0,54037.0,...,WV,WV,NaN,NaN,NaN,NaN,NaN,NaN,"[MD, PA, VA, WV, nan]",3
1133,MA,25003.0,9005.0,25003.0,25011.0,25013.0,25015.0,36021.0,36027.0,36083.0,...,NY,VT,NaN,NaN,NaN,NaN,NaN,NaN,"[CT, MA, NY, VT, nan]",3
1145,MA,25027.0,9013.0,9015.0,25011.0,25013.0,25015.0,25017.0,25021.0,25027.0,...,MA,NH,NH,RI,NaN,NaN,NaN,NaN,"[CT, MA, NH, RI, nan]",3
1663,NM,35045.0,4001.0,8007.0,8067.0,8083.0,35031.0,35039.0,35043.0,35045.0,...,NM,UT,NaN,NaN,NaN,NaN,NaN,NaN,"[AZ, CO, NM, UT, nan]",3
2536,UT,49037.0,4001.0,4005.0,4017.0,8033.0,8077.0,8083.0,8085.0,8113.0,...,CO,NM,UT,UT,UT,UT,UT,NaN,"[AZ, nan, CO, NM, UT]",3


In [104]:
def squeeze_nan(x):
    original_columns = x.index.tolist()

    squeezed = x.dropna()
    squeezed.index = [original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan)

In [105]:
tst = pd.DataFrame(neighbors['border_states'].to_list())
tst = tst.apply(squeeze_nan,axis=1)
tst = tst.dropna(how='all',axis=1)
state_brs = ['border_state1','border_state2','border_state3','border_state4']
tst.columns = state_brs
neighbors = pd.concat([neighbors,tst],axis=1)

In [106]:
df_all = df.merge(neighbors[['state_abb_x','FIPS','count_border_states','border_state1','border_state2','border_state3','border_state4']],on=['state_abb_x','FIPS'])

In [107]:
df_all[(df_all['count_border_states']>0)&(df_all['count_border_states'].notnull())]['state_x'].value_counts()

Georgia           3362
North Carolina    3305
Kentucky          3281
Tennessee         3216
Virginia          3210
Missouri          3148
Indiana           2873
Texas             2805
Illinois          2798
Iowa              2569
Mississippi       2430
Pennsylvania      2407
Wisconsin         2405
Ohio              2228
New York          2158
Arkansas          2152
Alabama           2111
Minnesota         2099
West Virginia     2087
Louisiana         1754
Michigan          1731
Oklahoma          1690
Idaho             1660
South Carolina    1618
Maryland          1577
Kansas            1509
Oregon            1354
South Dakota      1348
New Jersey        1328
Nebraska          1284
Colorado          1231
New Mexico        1212
Florida           1162
Wyoming            988
California         972
Montana            952
Washington         905
Utah               876
Nevada             767
Vermont            761
New Hampshire      664
Connecticut        664
Massachusetts      664
Arizona    

In [108]:
prez = prez_data()
df_all = df_all.merge(prez, on =['FIPS'],how='left')

In [109]:
orders = ['soe', 'sah', 'state_ease', 'state_sah',
'state_mandate', 'state_non_ess', 'state_lgb', 'state_sc',
'state_bar_res', 'state_election', 'state_emergency',
'state_compliance', 'state_prepare']
for x in orders:
    df_all['post_{}'.format(x)] = np.where(df_all[x]<=df_all['date'],1,0)

In [110]:
df_all[df_all['state_abb_x']=='WY']['sah']

222237    NaN
222238    NaN
222239    NaN
222240    NaN
222241    NaN
         ... 
223916    NaN
223917    NaN
223918    NaN
223919    NaN
223920    NaN
Name: sah, Length: 1684, dtype: object

In [111]:
states = pd.concat([df_all[['state_abb_x','date']],df_all.filter(like='post_')],axis=1)
states.drop_duplicates(inplace=True)
states.columns.values

array(['state_abb_x', 'date', 'post_soe', 'post_sah', 'post_state_ease',
       'post_state_sah', 'post_state_mandate', 'post_state_non_ess',
       'post_state_lgb', 'post_state_sc', 'post_state_bar_res',
       'post_state_election', 'post_state_emergency',
       'post_state_compliance', 'post_state_prepare'], dtype=object)

In [112]:
for i,x in enumerate(state_brs):
    i = i +1
    states.columns = [x, 'date', 'post_soe_n{}'.format(i), 'post_sah_n{}'.format(i), 'post_state_ease_n{}'.format(i),
       'post_state_sah_n{}'.format(i), 'post_state_mandate_n{}'.format(i), 'post_state_non_ess_n{}'.format(i),
       'post_state_lgb_n{}'.format(i), 'post_state_sc_n{}'.format(i), 'post_state_bar_res_n{}'.format(i),
       'post_state_election_n{}'.format(i), 'post_state_emergency_n{}'.format(i),
       'post_state_compliance_n{}'.format(i), 'post_state_prepare_n{}'.format(i)]
    df_all = df_all.merge(states, on=[x,'date'],how='left')

In [114]:
df_all[df_all['post_state_compliance_n4'].notnull()].iloc[100:150,:].to_csv('chk.csv')